In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip "/content/drive/My Drive/SyncPC/Deep learning/IMDB Dataset.csv.zip" 

Archive:  /content/drive/My Drive/SyncPC/Deep learning/IMDB Dataset.csv.zip
  inflating: IMDB Dataset.csv        


/content/drive/My Drive/Jaist/jaist/Toshiba-Project/GoogleNews-vectors-negative300.bin

In [ ]:
import pandas as pd
data_pd = pd.read_csv("IMDB Dataset.csv")

data_pd["label"] = data_pd["sentiment"].map({'positive': 1, 'negative': 0})
data_pd.head()

,review,sentiment,label
0,One of the other reviewers has mentioned that ...,positive,1
1,A wonderful little production. <br /><br />The...,positive,1
2,I thought this was a wonderful way to spend ti...,positive,1
3,Basically there's a family where a little boy ...,negative,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,1


In [ ]:
from keras.preprocessing.text import Tokenizer
vocab_size = 5000
t = Tokenizer(num_words=vocab_size)
# fit the tokenizer on the documents
t.fit_on_texts(data_pd["review"].values)


In [ ]:
t.word_index

{'the': 1,
 'and': 2,
 'a': 3,
 'of': 4,
 'to': 5,
 'is': 6,
 'br': 7,
 'in': 8,
 'it': 9,
 'i': 10,
 'this': 11,
 'that': 12,
 'was': 13,
 'as': 14,
 'for': 15,
 'with': 16,
 'movie': 17,
 'but': 18,
 'film': 19,
 'on': 20,
 'not': 21,
 'you': 22,
 'are': 23,
 'his': 24,
 'have': 25,
 'be': 26,
 'one': 27,
 'he': 28,
 'all': 29,
 'at': 30,
 'by': 31,
 'an': 32,
 'they': 33,
 'so': 34,
 'who': 35,
 'from': 36,
 'like': 37,
 'or': 38,
 'just': 39,
 'her': 40,
 'out': 41,
 'about': 42,
 'if': 43,
 "it's": 44,
 'has': 45,
 'there': 46,
 'some': 47,
 'what': 48,
 'good': 49,
 'when': 50,
 'more': 51,
 'very': 52,
 'up': 53,
 'no': 54,
 'time': 55,
 'my': 56,
 'even': 57,
 'would': 58,
 'she': 59,
 'which': 60,
 'only': 61,
 'really': 62,
 'see': 63,
 'story': 64,
 'their': 65,
 'had': 66,
 'can': 67,
 'me': 68,
 'well': 69,
 'were': 70,
 'than': 71,
 'much': 72,
 'we': 73,
 'bad': 74,
 'been': 75,
 'get': 76,
 'do': 77,
 'great': 78,
 'other': 79,
 'will': 80,
 'also': 81,
 'into': 82,
 'p

In [ ]:
list(t.word_index.keys())[0:10]

['the', 'and', 'a', 'of', 'to', 'is', 'br', 'in', 'it', 'i']

In [ ]:
from gensim.models.keyedvectors import KeyedVectors
import numpy as np
def loadWWordembedding(word2id,word2vec_path,vocab_size):
  model_org = KeyedVectors.load_word2vec_format(word2vec_path, binary=bin)
      
  #get W weight for embedding layer

  W = np.zeros(shape=(vocab_size, 300), dtype='float32')
  W[0] = np.zeros(300, dtype='float32')

  count_in = 0
  count_out = 0
  for word in list(word2id.keys())[0:vocab_size-1]:
    i = word2id[word]
    if word in model_org:
      W[i] = model_org[word]
      count_in += 1
    else:
      W[i] = np.random.uniform(-0.25,0.25,300)
      count_out += 1

  print (count_in, count_out)
  return W

In [ ]:
W = loadWWordembedding(t.word_index,"/content/drive/My Drive/Jaist/jaist/Toshiba-Project/GoogleNews-vectors-negative300.bin",vocab_size)

4863 136


In [ ]:
X = t.texts_to_sequences(data_pd["review"].values)
y = data_pd["label"].values

In [ ]:
from keras.preprocessing.sequence import pad_sequences
X = pad_sequences(X,maxlen=300)


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42, stratify=y)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)


print (X_train.shape,X_test.shape, X_val.shape)

(20000, 300) (25000, 300) (5000, 300)


In [ ]:
from keras.layers import *
from keras.models import Model
from keras.callbacks import ModelCheckpoint


input_layer = Input(shape=(300,))
emb_layer = Embedding(vocab_size,300,weights=[W])(input_layer)
conv_layer = Conv1D(100,3, activation="relu")(emb_layer)
pool_layer = GlobalAveragePooling1D()(conv_layer)
hidden_dense_layer = Dense(50,activation="sigmoid")(pool_layer)
dense_layer = Dense(1,activation="sigmoid")(hidden_dense_layer)

model = Model(inputs=input_layer, outputs=dense_layer)
model.compile(loss='binary_crossentropy', optimizer="adam",metrics=["accuracy"])
print (model.summary())
mc = ModelCheckpoint("best_checkpoint.h5", monitor="val_accuracy" , save_best_only=True, save_weights_only=True)

model.fit(X_train,y_train,validation_data=(X_val,y_val), epochs= 20, batch_size=50, callbacks=[mc])

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 300)]             0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 300, 300)          1500000   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 298, 100)          90100     
_________________________________________________________________
global_average_pooling1d_1 ( (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 51        
Total params: 1,595,201
Trainable params: 1,595,201
Non-trainable params: 0
_________________________________________________

In [ ]:
model.load_weights("best_checkpoint.h5")

y_test_pred = model.predict(X_test)
y_test_pred = [1 if pred > 0.5 else 0 for pred in y_test_pred ]
import numpy as np
print("Acc:",(1 - np.sum(np.abs(y_test_pred- y_test))/len(y_test_pred)))

Acc: 0.88636
